<a href="https://colab.research.google.com/github/shayari21/Udacity--Secure-AI/blob/master/Query_db_privacy_estimation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here, we query the entire database first, and then query it by element by element to measure the change in output.

In [4]:
pip install syft

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 450kB 44.6MB/s 
     |████████████████████████████████| 204kB 50.1MB/s 
     |████████████████████████████████| 71kB 26.8MB/s 
     |████████████████████████████████| 389kB 47.8MB/s 
     |████████████████████████████████| 1.4MB 39.9MB/s 
     |████████████████████████████████| 51kB 21.9MB/s 
     |████████████████████████████████| 276kB 45.4MB/s 
     |████████████████████████████████| 122kB 49.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import torch
import syft

def get_parallel_db(db,remove_index):
  
  
  return torch.cat((db[0:remove_index],db[remove_index+1:]))

def get_parallel_dbs(db):
  
  parallel_dbs = list()

  for i in range(len(db)):
    pdb=get_parallel_db(db,i)
    parallel_dbs.append(pdb)
    
  return parallel_dbs

def create_db_and_parallels(num_entries):
  
  db=torch.randn(num_entries) > 0.5
  pdbs=get_parallel_dbs(db)
  
  return db, pdbs

db , pdbs= create_db_and_parallels(20)
db

tensor([1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       dtype=torch.uint8)

In [6]:
pdbs

[tensor([0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,

In [0]:
#querying the entire database

def query(db):
  return db.sum()

full_db_result=query(db)

In [8]:
query(pdbs[15]) # REAMAINS SAME!

tensor(6)

In [9]:
#Querying on every parallel databases

sensitivity=0

for pdb in pdbs:
  pdb_result=query(pdb)
  
  db_distance=torch.abs(pdb_result-full_db_result)
  
  if(db_distance>sensitivity):
    sensitivity= db_distance
  
sensitivity
  

tensor(1)

**Sensitivity**  refers to the max amount of query change when an individual is removed. It is allso called simple/L1 sensitivity.



In [10]:
def sensitivity(query,n_entries):
  db , pdbs= create_db_and_parallels(n_entries)
  full_db_result=query(db)
  
  max_d=0
  
  for pdb in pdbs:
    pdb_result=query(pdb)
  
    db_distance=torch.abs(pdb_result-full_db_result)
  
    
    if(db_distance>max_d):
      max_d= db_distance
      
    return max_d
  
def query(db):
  return db.float().mean()

sensitivity(query,1000)

tensor(0.0003)

#Calculating L1 sensitivity of databases

Here, we calculate emperical sensitivity of the databases. We define the term **threshold** to calculate the sensitvity. Thougn this sensitivity is not valid when applied to data conditioned sensitivity. In Data conditional sensitivity, we consider the range as well as the values, which we do not consider in the following code:


In [16]:
def query(db, threshold=5):
  return(db.sum() > threshold).float()

db, pdbs= create_db_and_parallels(10)
db.sum()

for i in range(10):
  sens_f= sensitivity(query,n_entries=10)
  print(sens_f)


0
0
tensor(1.)
0
0
0
0
0
0
0


# Performing Differencing attack on column 10

In [19]:
db,_= create_db_and_parallels(100)

pdb= get_parallel_db(db, remove_index=10)

#differencing attack with sum( can be done using mean and threshold concept)
sum(db)-sum(pdb)

tensor(1, dtype=torch.uint8)